Import Library

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc

from sklearn.impute import SimpleImputer # Untuk imputasi missing value
from sklearn.ensemble import IsolationForest # Untuk deteksi outlier
from scipy.stats import zscore # Untuk deteksi outlier berbasis Z-score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


### Data Processing

Optimasi Memory

In [2]:
# --- Fungsi Utility ---
def reduce_mem_usage(df):
    """ Iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else: # float types
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


Handle Missing Values & Outliers

In [3]:
# Handle missing value & outliers

def handle_specific_outliers(df, df_name):
    """ Handles specific known outliers like DAYS_EMPLOYED anomaly. """
    if 'DAYS_EMPLOYED' in df.columns:
        print(f"  Handling specific outliers for DAYS_EMPLOYED in {df_name}...")
        df['DAYS_EMPLOYED_ANOM'] = df["DAYS_EMPLOYED"] == 365243
        df['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)
        print(f"    Created DAYS_EMPLOYED_ANOM and replaced 365243 with NaN.")
    return df

def handle_missing_values(df, df_name):
    """ Fills missing values and potentially drops columns with too many NaNs. """
    print(f"  Handling missing values in {df_name}...")
    threshold = 0.7 * len(df)
    cols_to_drop = [col for col in df.columns if df[col].isnull().sum() > threshold]
    if cols_to_drop:
        print(f"    Dropping {len(cols_to_drop)} columns with more than 70% missing values in {df_name}.")
        df.drop(columns=cols_to_drop, inplace=True)

    # Separate numerical and categorical columns for imputation
    numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = df.select_dtypes(include='category').columns.tolist()

    # Impute numerical columns with median, excluding IDs and TARGET
    cols_to_impute_num = [col for col in numerical_cols if col not in ['SK_ID_CURR', 'SK_ID_PREV', 'SK_ID_BUREAU', 'TARGET']] 
    if cols_to_impute_num:
        imputer_num = SimpleImputer(strategy='median')
        df[cols_to_impute_num] = imputer_num.fit_transform(df[cols_to_impute_num])
        print(f"    Imputed {len(cols_to_impute_num)} numerical columns with median.")

    # Impute categorical columns with mode
    if categorical_cols:
        imputer_cat = SimpleImputer(strategy='most_frequent')
        df[categorical_cols] = imputer_cat.fit_transform(df[categorical_cols])
        print(f"    Imputed {len(categorical_cols)} categorical columns with mode.")

    return df

def handle_all_numeric_outliers(df, df_name, id_cols=None):
    """ Caps outliers for all numerical columns using IQR method (1% and 99% percentiles). """
    print(f"  Handling numerical outliers in {df_name} using 1st and 99th percentile capping...")
    numerical_cols = df.select_dtypes(include=np.number).columns.tolist()

    if id_cols is None:
        id_cols = ['SK_ID_CURR'] 
    # Exclude ID columns and TARGET from outlier capping
    cols_to_cap = [col for col in numerical_cols if col not in id_cols and col != 'TARGET']

    for col in cols_to_cap:
        # Check if column has enough non-NaN values to calculate percentiles
        if df[col].count() > 0:
            lower_bound = df[col].quantile(0.01)
            upper_bound = df[col].quantile(0.99)
            df[col] = np.clip(df[col], lower_bound, upper_bound)
    print(f"    Capped outliers for {len(cols_to_cap)} numerical columns.")
    return df

Feature Engineering (Agregasi Data), Transformasi Fitur (Encoding), Final Data Preparation (Final Missing Values Check, StandardScaler)

In [4]:
def process_and_aggregate_data():
    print("\n--- Loading main application data ---")
    app_train = pd.read_csv('application_train.csv')
    app_test = pd.read_csv('application_test.csv')

    print(f"Initial app_train shape: {app_train.shape}")
    print(f"Initial app_test shape: {app_test.shape}")

    # Tambahkan kolom 'TARGET' ke app_test dengan semua nilai NaN
    app_test['TARGET'] = np.nan

    # Gabungkan train dan test
    df = pd.concat([app_train, app_test], ignore_index=True)
    del app_train, app_test 
    gc.collect()

    print(f"Combined df shape after concat: {df.shape}")

    # Process combined application data
    df = reduce_mem_usage(df)
    df = handle_specific_outliers(df, 'Application Data')
    df = handle_missing_values(df, 'Application Data')
    df = handle_all_numeric_outliers(df, 'Application Data', id_cols=['SK_ID_CURR', 'TARGET'])
    gc.collect()

    # --- Process Bureau and Bureau Balance ---
    print("\n--- Processing Bureau and Bureau Balance ---")
    bureau = pd.read_csv('bureau.csv')
    bureau = reduce_mem_usage(bureau)
    bureau = handle_missing_values(bureau, 'Bureau Data')
    bureau = handle_all_numeric_outliers(bureau, 'Bureau Data', id_cols=['SK_ID_CURR', 'SK_ID_BUREAU'])

    bureau_balance = pd.read_csv('bureau_balance.csv')
    bureau_balance = reduce_mem_usage(bureau_balance)
    bureau_balance = handle_missing_values(bureau_balance, 'Bureau Balance Data') # No specific outliers for BB

    print("  Aggregating bureau_balance data...")
    # Debugging: Periksa kondisi bureau_balance sebelum agregasi
    print(f"  Shape of bureau_balance AFTER cleaning: {bureau_balance.shape}")
    print(f"  Unique SK_ID_BUREAU in bureau_balance: {bureau_balance['SK_ID_BUREAU'].nunique()}")
    print(f"  Is SK_ID_BUREAU present in bureau_balance? {'SK_ID_BUREAU' in bureau_balance.columns}")
    print(f"  Data types of bureau_balance: \n{bureau_balance.dtypes}")
    print(f"  Categorical columns in bureau_balance: {bureau_balance.select_dtypes(include='category').columns.tolist()}")
    print(f"  Numerical columns in bureau_balance: {bureau_balance.select_dtypes(include=np.number).columns.tolist()}")

    bb_numerical_cols = bureau_balance.select_dtypes(include=np.number).columns.drop(['SK_ID_BUREAU'], errors='ignore').tolist()
    bb_categorical_cols = bureau_balance.select_dtypes(include='category').columns.tolist()

    bb_agg_dict = {}
    # Agregasi kolom numerik (MONTHS_BALANCE adalah yang utama di sini)
    for col in bb_numerical_cols:
        bb_agg_dict[col] = ['min', 'max', 'mean', 'sum', 'count', 'nunique']

    # Agregasi kolom kategorikal (STATUS adalah yang utama di sini)
    for col in bb_categorical_cols:
        bb_agg_dict[col] = ['count', 'nunique', (f'{col}_mode', lambda x: x.mode()[0] if not x.mode().empty else np.nan)]

    # Periksa apakah dictionary agregasi kosong. Penting agar .agg() tidak error.
    if not bb_agg_dict:
        print("WARNING: Aggregation dictionary for bureau_balance is empty. This might indicate no relevant columns were found.")
        bureau_balance_agg = pd.DataFrame({'SK_ID_BUREAU': bureau_balance['SK_ID_BUREAU'].unique()})
    else:
        bureau_balance_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(bb_agg_dict).reset_index()

    # Flatten multi-level columns
    new_cols = ['SK_ID_BUREAU']
    if not bureau_balance_agg.empty: # Only process if DataFrame is not empty
        for col_tuple in bureau_balance_agg.columns.drop('SK_ID_BUREAU'):
            if isinstance(col_tuple, tuple):
                if col_tuple[1] == '<lambda>':
                    new_cols.append(f'BB_{col_tuple[0].upper()}_{col_tuple[1].upper().replace("<LAMBDA>", "MODE")}')
                else:
                    new_cols.append(f'BB_{col_tuple[0].upper()}_{col_tuple[1].upper()}')
            else:
                new_cols.append(f'BB_{str(col_tuple).upper()}')
                
        bureau_balance_agg.columns = new_cols
    del bureau_balance
    gc.collect()
    print(f"  Aggregated bureau_balance data. Shape: {bureau_balance_agg.shape}")

    # Merge aggregated bureau_balance to bureau
    bureau = bureau.merge(bureau_balance_agg, on='SK_ID_BUREAU', how='left')
    del bureau_balance_agg
    gc.collect()
    print(f"  Merged bureau_balance to bureau. Bureau shape: {bureau.shape}")

    print("  Aggregating bureau data...")
    # Aggregate bureau data to SK_ID_CURR level
    bureau_numerical_cols = bureau.select_dtypes(include=np.number).columns.drop(['SK_ID_CURR', 'SK_ID_BUREAU'], errors='ignore').tolist()
    bureau_categorical_cols = bureau.select_dtypes(include='category').columns.tolist()

    bureau_agg_dict = {}
    for col in bureau_numerical_cols:
        bureau_agg_dict[col] = ['min', 'max', 'mean', 'sum', 'std', 'count', 'median', 'first', 'last']
    for col in bureau_categorical_cols:
        bureau_agg_dict[col] = ['nunique', (f'{col}_mode', lambda x: x.mode()[0] if not x.mode().empty else np.nan)]

    bureau_agg = bureau.groupby('SK_ID_CURR').agg(bureau_agg_dict).reset_index()

    # Flatten multi-level columns
    new_cols = ['SK_ID_CURR']
    for col_tuple in bureau_agg.columns.drop('SK_ID_CURR'):
        if isinstance(col_tuple, tuple):
            if col_tuple[1] == '<lambda>':
                new_cols.append(f'BUREAU_{col_tuple[0].upper()}_{col_tuple[1].upper().replace("<LAMBDA>", "MODE")}')
            else:
                new_cols.append(f'BUREAU_{col_tuple[0].upper()}_{col_tuple[1].upper()}')
        else:
             new_cols.append(f'BUREAU_{str(col_tuple).upper()}')

    bureau_agg.columns = new_cols
    del bureau
    gc.collect()
    print(f"  Aggregated bureau data. Shape: {bureau_agg.shape}")

    # Merge aggregated bureau data to main df
    df = df.merge(bureau_agg, on='SK_ID_CURR', how='left')
    del bureau_agg
    gc.collect()
    print(f"Merged bureau data to main df. Main df shape: {df.shape}")

    # --- Process Previous Application ---
    print("\n--- Processing Previous Application ---")
    previous_application = pd.read_csv('previous_application.csv')
    previous_application = reduce_mem_usage(previous_application)
    previous_application = handle_missing_values(previous_application, 'Previous Application Data')
    previous_application = handle_all_numeric_outliers(previous_application, 'Previous Application Data', id_cols=['SK_ID_CURR', 'SK_ID_PREV'])

    print("  Aggregating previous_application data...")
    # Aggregate previous_application
    prev_num_cols = previous_application.select_dtypes(include=np.number).columns.drop(['SK_ID_CURR', 'SK_ID_PREV'], errors='ignore').tolist()
    prev_cat_cols = previous_application.select_dtypes(include='category').columns.tolist()

    prev_agg_dict = {}
    for col in prev_num_cols:
        prev_agg_dict[col] = ['min', 'max', 'mean', 'sum', 'std', 'count', 'median']
    for col in prev_cat_cols:
        prev_agg_dict[col] = ['nunique', (f'{col}_mode', lambda x: x.mode()[0] if not x.mode().empty else np.nan)]

    previous_application_agg = previous_application.groupby('SK_ID_CURR').agg(prev_agg_dict).reset_index()
    # Flatten multi-level columns
    new_cols = ['SK_ID_CURR']
    for col_tuple in previous_application_agg.columns.drop('SK_ID_CURR'):
        if isinstance(col_tuple, tuple):
            if col_tuple[1] == '<lambda>':
                new_cols.append(f'PREV_{col_tuple[0].upper()}_{col_tuple[1].upper().replace("<LAMBDA>", "MODE")}')
            else:
                new_cols.append(f'PREV_{col_tuple[0].upper()}_{col_tuple[1].upper()}')
        else:
            new_cols.append(f'PREV_{str(col_tuple).upper()}')

    previous_application_agg.columns = new_cols
    del previous_application
    gc.collect()
    print(f"  Aggregated previous_application data. Shape: {previous_application_agg.shape}")

    df = df.merge(previous_application_agg, on='SK_ID_CURR', how='left')
    del previous_application_agg
    gc.collect()
    print(f"Merged previous_application data to main df. Main df shape: {df.shape}")


    # --- Process POS_CASH_balance ---
    print("\n--- Processing POS_CASH_balance ---")
    pos_cash = pd.read_csv('POS_CASH_balance.csv')
    pos_cash = reduce_mem_usage(pos_cash)
    pos_cash = handle_missing_values(pos_cash, 'POS_CASH_balance Data')
    pos_cash = handle_all_numeric_outliers(pos_cash, 'POS_CASH_balance Data', id_cols=['SK_ID_CURR', 'SK_ID_PREV'])

    print("  Aggregating POS_CASH_balance data...")
    # Aggregate POS_CASH_balance
    pos_cash_num_cols = pos_cash.select_dtypes(include=np.number).columns.drop(['SK_ID_CURR', 'SK_ID_PREV'], errors='ignore').tolist()
    pos_cash_cat_cols = pos_cash.select_dtypes(include='category').columns.tolist()

    pos_cash_agg_dict = {}
    for col in pos_cash_num_cols:
        pos_cash_agg_dict[col] = ['min', 'max', 'mean', 'sum', 'std', 'count', 'median']
    for col in pos_cash_cat_cols:
        pos_cash_agg_dict[col] = ['nunique', (f'{col}_mode', lambda x: x.mode()[0] if not x.mode().empty else np.nan)]

    pos_cash_agg = pos_cash.groupby('SK_ID_CURR').agg(pos_cash_agg_dict).reset_index()
    # Flatten multi-level columns
    new_cols = ['SK_ID_CURR']
    for col_tuple in pos_cash_agg.columns.drop('SK_ID_CURR'):
        if isinstance(col_tuple, tuple):
            if col_tuple[1] == '<lambda>':
                new_cols.append(f'POS_CASH_{col_tuple[0].upper()}_{col_tuple[1].upper().replace("<LAMBDA>", "MODE")}')
            else:
                new_cols.append(f'POS_CASH_{col_tuple[0].upper()}_{col_tuple[1].upper()}')
        else:
            new_cols.append(f'POS_CASH_{str(col_tuple).upper()}')

    pos_cash_agg.columns = new_cols
    del pos_cash
    gc.collect()
    print(f"  Aggregated POS_CASH_balance data. Shape: {pos_cash_agg.shape}")

    df = df.merge(pos_cash_agg, on='SK_ID_CURR', how='left')
    del pos_cash_agg
    gc.collect()
    print(f"Merged POS_CASH_balance data to main df. Main df shape: {df.shape}")

    # --- Process Credit Card Balance ---
    print("\n--- Processing Credit Card Balance ---")
    credit_card = pd.read_csv('credit_card_balance.csv')
    credit_card = reduce_mem_usage(credit_card)
    credit_card = handle_missing_values(credit_card, 'Credit Card Balance Data')
    credit_card = handle_all_numeric_outliers(credit_card, 'Credit Card Balance Data', id_cols=['SK_ID_CURR', 'SK_ID_PREV'])

    print("  Aggregating credit_card_balance data...")

    credit_card_numerical_cols = credit_card.select_dtypes(include=np.number).columns.drop(['SK_ID_CURR', 'SK_ID_PREV'], errors='ignore').tolist()
    credit_card_categorical_cols = credit_card.select_dtypes(include='category').columns.tolist()

    credit_card_agg_dict = {}
    for col in credit_card_numerical_cols:
        credit_card_agg_dict[col] = ['min', 'max', 'mean', 'sum', 'std', 'count', 'median']
    for col in credit_card_categorical_cols:
        credit_card_agg_dict[col] = ['nunique', (f'{col}_mode', lambda x: x.mode()[0] if not x.mode().empty else np.nan)]

    credit_card_agg = credit_card.groupby('SK_ID_CURR').agg(credit_card_agg_dict).reset_index()

    new_cols = ['SK_ID_CURR']
    for col_tuple in credit_card_agg.columns.drop('SK_ID_CURR'):
        if isinstance(col_tuple, tuple):
            if col_tuple[1] == '<lambda>':
                new_cols.append(f'CC_BAL_{col_tuple[0].upper()}_{col_tuple[1].upper().replace("<LAMBDA>", "MODE")}')
            else:
                new_cols.append(f'CC_BAL_{col_tuple[0].upper()}_{col_tuple[1].upper()}')
        else:
             new_cols.append(f'CC_BAL_{str(col_tuple).upper()}')

    credit_card_agg.columns = new_cols
    del credit_card
    gc.collect()
    print(f"  Aggregated credit_card_balance data. Shape: {credit_card_agg.shape}")

    df = df.merge(credit_card_agg, on='SK_ID_CURR', how='left')
    del credit_card_agg
    gc.collect()
    print(f"Merged credit_card_balance data to main df. Main df shape: {df.shape}")


    # --- Process Installments Payments ---
    print("\n--- Processing Installments Payments ---")
    installments_payments = pd.read_csv('installments_payments.csv')
    installments_payments = reduce_mem_usage(installments_payments)
    installments_payments = handle_missing_values(installments_payments, 'Installments Payments Data')
    installments_payments = handle_all_numeric_outliers(installments_payments, 'Installments Payments Data', id_cols=['SK_ID_CURR', 'SK_ID_PREV'])

    print("  Aggregating installments_payments data...")
    # Aggregate installments_payments
    inst_num_cols = installments_payments.select_dtypes(include=np.number).columns.drop(['SK_ID_CURR', 'SK_ID_PREV'], errors='ignore').tolist()
    inst_cat_cols = installments_payments.select_dtypes(include='category').columns.tolist()

    inst_agg_dict = {}
    for col in inst_num_cols:
        inst_agg_dict[col] = ['min', 'max', 'mean', 'sum', 'std', 'count', 'median']
    for col in inst_cat_cols:
        inst_agg_dict[col] = ['nunique', (f'{col}_mode', lambda x: x.mode()[0] if not x.mode().empty else np.nan)]

    installments_payments_agg = installments_payments.groupby('SK_ID_CURR').agg(inst_agg_dict).reset_index()
    # Flatten multi-level columns
    new_cols = ['SK_ID_CURR']
    for col_tuple in installments_payments_agg.columns.drop('SK_ID_CURR'):
        if isinstance(col_tuple, tuple):
            if col_tuple[1] == '<lambda>':
                new_cols.append(f'INSTAL_{col_tuple[0].upper()}_{col_tuple[1].upper().replace("<LAMBDA>", "MODE")}')
            else:
                new_cols.append(f'INSTAL_{col_tuple[0].upper()}_{col_tuple[1].upper()}')
        else:
            new_cols.append(f'INSTAL_{str(col_tuple).upper()}')

    installments_payments_agg.columns = new_cols
    del installments_payments
    gc.collect()
    print(f"  Aggregated installments_payments data. Shape: {installments_payments_agg.shape}")

    df = df.merge(installments_payments_agg, on='SK_ID_CURR', how='left')
    del installments_payments_agg
    gc.collect()
    print(f"Merged installments_payments data to main df. Main df shape: {df.shape}")

    # --- Final Data Transformation on the Combined DataFrame ---
    print("\n=======================================================")
    print("Starting Final Data Transformation (Encoding & Scaling)")
    print("=======================================================")

    # Handle categorical features (One-Hot Encoding for multi-valued categories)
    # and Label Encoding for binary categories
    print("  Encoding categorical features...")
    all_categorical_cols = df.select_dtypes(include='category').columns.tolist()

    # Pisahkan kolom biner dan multi-kelas
    binary_cat_cols = [col for col in all_categorical_cols if df[col].nunique() == 2]
    multi_cat_cols = [col for col in all_categorical_cols if df[col].nunique() > 2]

    # Label Encode fitur kategorikal biner
    for col in binary_cat_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        print(f"    Label encoded: {col}")

    # One-Hot Encode fitur kategorikal multi-kelas
    if multi_cat_cols:
        print(f"    One-Hot Encoding {len(multi_cat_cols)} multi-class categorical features...")
        df = pd.get_dummies(df, columns=multi_cat_cols, dummy_na=False)
        print(f"    DataFrame shape after One-Hot Encoding: {df.shape}")
    else:
        print("    No multi-class categorical features to One-Hot Encode.")
    gc.collect()

    print("  Handling remaining missing values with 0 and infinite values...")

    # Identifikasi kolom numerik yang mungkin masih memiliki NaN setelah semua merge dan encoding
    numerical_cols_for_final_fill = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col]) and col not in ['SK_ID_CURR', 'TARGET']]

    # Isi NaN dengan 0 untuk kolom-kolom ini
    df[numerical_cols_for_final_fill] = df[numerical_cols_for_final_fill].fillna(0)
    print(f"    Filled all remaining numerical NaNs with 0 for {len(numerical_cols_for_final_fill)} columns.")

    # Ganti nilai infinite dengan 0
    df.replace([np.inf, -np.inf], 0, inplace=True)
    print("    Replaced infinite values with 0.")

    # Optional: Verifikasi apakah masih ada NaN yang tersisa
    if df.isnull().any().any():
        print("WARNING: Some NaNs still exist after final fillna(0). Check these columns:")
        print(df.isnull().sum()[df.isnull().sum() > 0])


    print(f"  Final DataFrame shape after all preprocessing (before splitting): {df.shape}")

    print("  Re-optimizing memory usage before splitting and scaling...")
    df = reduce_mem_usage(df) # Jalankan lagi!
    gc.collect()
    
    # Separate train and test data
    train_df = df[df['TARGET'].notna()].copy()
    test_df = df[df['TARGET'].isna()].drop(columns=['TARGET']).copy()

    print(f"\nTrain DataFrame shape AFTER SPLIT: {train_df.shape}")
    print(f"Test DataFrame shape AFTER SPLIT: {test_df.shape}")

    # Pastikan test_df memiliki SK_ID_CURR untuk submission
    if 'SK_ID_CURR' not in test_df.columns:
        print("WARNING: SK_ID_CURR not found in test_df after splitting. This might cause submission issues.")

    del df
    gc.collect()

    # Apply StandardScaler to numerical features
    print("  Applying StandardScaler to numerical features...")
    scaler = StandardScaler()
    common_numerical_cols = [col for col in train_df.columns if col in test_df.columns and col not in ['SK_ID_CURR', 'TARGET'] and pd.api.types.is_numeric_dtype(train_df[col])]

    train_df[common_numerical_cols] = scaler.fit_transform(train_df[common_numerical_cols].astype(np.float32))
    test_df[common_numerical_cols] = scaler.transform(test_df[common_numerical_cols].astype(np.float32))
    print(f"  Scaled {len(common_numerical_cols)} numerical features.")
    gc.collect() 
    
    print(f"  Scaled {len(common_numerical_cols)} numerical features.")

    return train_df, test_df


In [ ]:
# --- Cara Memanggil Fungsi (Uncomment untuk menjalankan) ---
train_transformed, test_transformed = process_and_aggregate_data()

# # Tampilkan hasil (opsional, untuk verifikasi)
print("\n--- Transformed Train Data Head (Sample) ---")
print(train_transformed.head())
print("\n--- Transformed Test Data Head (Sample) ---")
print(test_transformed.head())


--- Loading main application data ---
Initial app_train shape: (307511, 122)
Initial app_test shape: (48744, 121)
Combined df shape after concat: (356255, 122)
Memory usage of dataframe is 331.60 MB
Memory usage after optimization is: 69.32 MB
Decreased by 79.1%
  Handling specific outliers for DAYS_EMPLOYED in Application Data...
    Created DAYS_EMPLOYED_ANOM and replaced 365243 with NaN.
  Handling missing values in Application Data...


Result Data Processing

In [ ]:
# Menyimpan
train_transformed.to_csv('train_transformed.csv', index=False)
test_transformed.to_csv('test_transformed.csv', index=False)